# Lesson 2: Classes in the Wild

> _Disclaimer: These points were intended to be applied to Python, your mileage may vary._

- Overutilising or underutilising classes can lead to ruin
- Classes can be a powerful tool or an endless garden path

## Pros

- Can keep track of state
  - No need to pass parameters back and forth
  - No thread-unsafe global variables
  - Can logically initialise state and then use it
- Can organise a hierarcy of states that belong together
- Provide dot-methods for accessing properties
  - "ask, don't tell"
  
## Cons

- Can make code convoluted and hard to read
  - code spread across multiple files
  - logic for a single operation spread across multiple different parts

## Functions vs Classes

### Functions vs Methods

In python

- a **function** takes parameters, returns a value
- a **method** can be called on an object, and can access state in the object

## Baby steps

In [1]:
CONFIG = {
    'thing': 'a',
    'identifiier': 'b',
    'name': 'c'
}

_This isn't very safe if something goes wrong, and the IDE can't offer us any help_

In [2]:
CONFIG['identifier']

KeyError: 'identifier'

## Namedtuple

`namedtuple` == Quick 'n' dirty class!

Use when you just need to
- make sure that the correct keys/values are present
- access something a few times (safely) via a dot method rather than a dict key lookup

In [2]:
from collections import namedtuple

Config = namedtuple('config', ['thing', 'identifier', 'name'])

CONFIG = Config('a', 'b', 'c')

print(CONFIG)
print(CONFIG.thing, CONFIG.identifier)

config(thing='a', identifier='b', name='c')
a b


### A multi-level config object

In [3]:
from collections import namedtuple

# Define what your config objects need to contain
Endpoint = namedtuple('endpoints', ['key', 'url', 'timeout', 'n_workers'])
Endpoints = namedtuple('endpoints', ['customers', 'products'])

# Initialise all configs with their values
# Could read this from a JSON file, command-line args, or define it here
# Either way, the namedtuple will ensure that the result is the same
ENDPOINTS = {
    'customers': Endpoint('customers', 'customers/search/', 200, 2),
    'products': Endpoint('products', 'products/all/search/', 100, 2)
}

for endpoint, config in ENDPOINTS.items():
    print(endpoint, config.url)

customers customers/search/
products products/all/search/


Now let's try the failing example again

In [15]:
Config(**{
    'thing': 'a',
    'identifiier': 'b',
    'name': 'c'
})

TypeError: <lambda>() got an unexpected keyword argument 'identifiier'

Much better!
This means that we catch the error when Config is _**initialised**_, rather than when trying to _**access**_ 'identifier' later on.

This is also useful when loading a JSON config, and you need to make sure all the key are present

In [16]:
import json
raw = '{"identifier": 123, "name": "me", "thing": 123}'

Config(**json.loads(raw))

config(thing=123, identifier=123, name='me')

In [17]:
raw = '{"identifier": 123, "name": "me", "thing": 123, "extra": 1}'

Config(**json.loads(raw))

TypeError: <lambda>() got an unexpected keyword argument 'extra'

## Dataclasses

- Python 3.7+
- Syntactic sugar for defining an `__init__` method and instance variables
- also provides a nice `__repr__` method, and some other things

A regular class:

In [6]:
class Obj:
    def __init__(self, a=1, b=2, c='default'):
        self.a = a
        self.b = b
        self.c = c

Obj(1)

The same, but as a dataclass

In [5]:
from dataclasses import dataclass

@dataclass
class Obj:
    a: int = 1
    b: int = 2
    c: str = 'default'

Obj(1)

Obj(a=1, b=2, c='default')

In [20]:
Obj(d=5)

TypeError: __init__() got an unexpected keyword argument 'd'

---

## An Example

- You have a collection of items, in this case ids and emails
- Need to iterate through them, collect some values, and pass them on

In [8]:
from itertools import islice
from faker import Faker
from utils import ppd

fake = Faker()

def fake_record(i, spanner=False):
    if spanner and i % 5 == 0:
        return (i, fake.uuid4().split('-')[0], fake.email(), None)
    else:
        return (i, fake.uuid4().split('-')[0], fake.email(), fake.pyint())

def iterate(n=10, spanner=False):
    '''
    This method will yield a tuple of each item, and a boolean indicating 
    if there are more items.
    After all items are consumed, this method will yield None/False
    (This means that we can't just do "for item in iterate(collection)")
    '''
    for i in range(n-1):
        yield fake_record(i, spanner), True
    yield fake_record(i, spanner), False
    while True:
        yield None, False

In [10]:
collection = iterate()
for i, el in enumerate(collection):
    print(el)
    if i >= 12:
        break

((0, 'ed2f75fb', 'edward92@gmail.com', 6440), True)
((1, '73355c12', 'joseph26@hamilton.com', 6975), True)
((2, '70177ecf', 'kleindanielle@gmail.com', 4390), True)
((3, '46284377', 'barkerlisa@brown.com', 7283), True)
((4, '6d624aaf', 'wbailey@hunter.com', 2649), True)
((5, 'f5a5145c', 'kcole@gmail.com', 6259), True)
((6, '0194e5d1', 'uavila@yahoo.com', 5547), True)
((7, '92055157', 'carolreyes@gmail.com', 4563), True)
((8, '44da408f', 'kylemanning@gmail.com', 2572), True)
((8, '5a2eb161', 'wyang@gmail.com', 8797), False)
(None, False)
(None, False)
(None, False)


---

Let's add some mandatory conditional logic that evaluates whether a result is useful

_For our exercise, we are only going to collect the ID if the last column (int) value is even_

In [11]:
collection = iterate()

ids = []
for el, has_next in collection:
    print(-1, el, has_next)
    if el[3] % 2:
        ids.append(el[1])
    if not has_next:
        break

-1 (0, '466a2f56', 'zmartin@gmail.com', 8214) True
-1 (1, '1793d38c', 'qbutler@gmail.com', 4337) True
-1 (2, '4e3ec023', 'shaneconley@anderson-west.biz', 8296) True
-1 (3, 'aa28bcca', 'eric05@yahoo.com', 1243) True
-1 (4, '60e442af', 'apierce@johnson.info', 7344) True
-1 (5, '7c8001d1', 'oweber@gmail.com', 4316) True
-1 (6, '10dfbb18', 'reyesjose@yahoo.com', 1595) True
-1 (7, '14351687', 'rebeccaharrison@foster-garcia.org', 6551) True
-1 (8, 'ac9b6bbb', 'combsjonathan@gmail.com', 3067) True
-1 (8, '36c7c610', 'megan17@gmail.com', 9140) False


### Adding more stuff

Let's add
- some details around how many items we consumed/are up to
- a string representation method

In [12]:
def ppr(i=-1, el=[], has_next=False):
    if el is None:
        print(i, el, has_next)
    else:
        print('{:<2d}{:<2} {:} {:<30} {:<5} {:}'.format(i, *list(map(str, el)), str(has_next)))

def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(collection):
        ppr(i, el, has_next)
        if el[3] % 2:
            ids.append(el[1])
        if has_next == False:
            break
    print(f'total items: {len(ids)}')
    return ids

## Problems

Let's add a spanner

In [13]:
process(iterate(10, True))

0 0  db9260ee ggarcia@weeks.com              None  True


TypeError: unsupported operand type(s) for %: 'NoneType' and 'int'

OK, so let's just add an `isinstance` check

In [15]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(collection):
        ppr(i, el, has_next)
        if isinstance(el[3], int) and el[3] % 2:
            ids.append(el[1])
        if has_next == False:
            break
    print(f'total items: {len(ids)}')
    return ids

ppd(process(iterate()))

0 0  cba8b99d cbullock@hotmail.com           5190  True
1 1  3cc038ea yolandacampbell@gmail.com      282   True
2 2  031f7f85 christopherwilliams@gmail.com  6796  True
3 3  56f715dc adamsamanda@hotmail.com        445   True
4 4  50357d2e ortegasteven@yahoo.com         9190  True
5 5  6705d82e hendersoncathy@hotmail.com     2835  True
6 6  6209de26 suzannekelly@robinson-paul.com 3764  True
7 7  d9942801 eric17@reynolds-cowan.com      3400  True
8 8  ad6532eb yvette29@fernandez-durham.biz  4302  True
9 8  0908389c milleramanda@cabrera.biz       6420  False
total items: 2
[
  "56f715dc",
  "6705d82e"
]



Also, we now want to skip and warn if any records have an email > 20 chars

In [16]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(collection):
        ppr(i, el, has_next)
        if isinstance(el, int) and el[3] % 2:
            ids.append(el[1])
        if len(el[2]) > 20:
            print(f'!! skipping {i}: {el}\n')
        if has_next == False:
            break
    print(f'total items: {len(ids)}')
    return ids

process(iterate())

0 0  30e3024b brandihickman@jones.com        7034  True
!! skipping 0: (0, '30e3024b', 'brandihickman@jones.com', 7034)

1 1  4f101ac0 christina18@williams-campos.info 8105  True
!! skipping 1: (1, '4f101ac0', 'christina18@williams-campos.info', 8105)

2 2  65e395f1 taylorstephen@bell-cohen.info  4342  True
!! skipping 2: (2, '65e395f1', 'taylorstephen@bell-cohen.info', 4342)

3 3  80e5725a glenngross@nelson.info         3568  True
!! skipping 3: (3, '80e5725a', 'glenngross@nelson.info', 3568)

4 4  16850fc5 brianbarrett@gmail.com         7460  True
!! skipping 4: (4, '16850fc5', 'brianbarrett@gmail.com', 7460)

5 5  8107d8c1 vhardy@savage.com              1493  True
6 6  dd712322 andrewsjames@rice.com          9511  True
!! skipping 6: (6, 'dd712322', 'andrewsjames@rice.com', 9511)

7 7  909b94ec timothy44@evans.net            8930  True
8 8  76f7d387 nicholebarrett@gmail.com       8464  True
!! skipping 8: (8, '76f7d387', 'nicholebarrett@gmail.com', 8464)

9 8  57a3503e lauren27@hotm

[]

It looks confusing now so let's add some comments

In [17]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(collection): # <--- ⚠
        ppr(i, el, has_next)
        # collect item if it's even
        if isinstance(el, int) and el[3] % 2: # < ----------------⚠
            ids.append(el[1])
        # Warn about large items
        if len(el[2]) > 20: # <-----------------------------------⚠
            print(f'!! {el}') # <-------------⚠
        if has_next == False:
            break
    print(f'total items: {len(ids)}')
    return ids

ppd(process(iterate()))

0 0  c43a5a35 nlowery@thompson-blair.com     3627  True
!! (0, 'c43a5a35', 'nlowery@thompson-blair.com', 3627)
1 1  4ecaa651 brendan41@yahoo.com            9548  True
2 2  735fc4fa susanwilliams@gmail.com        316   True
!! (2, '735fc4fa', 'susanwilliams@gmail.com', 316)
3 3  7bc3d64f stephanie04@cochran-miller.com 2355  True
!! (3, '7bc3d64f', 'stephanie04@cochran-miller.com', 2355)
4 4  9c0f5d89 priscillayang@freeman.net      9272  True
!! (4, '9c0f5d89', 'priscillayang@freeman.net', 9272)
5 5  062a6382 wendy25@mullins.com            1171  True
6 6  5dc7ca99 shane18@hull-reynolds.com      8306  True
!! (6, '5dc7ca99', 'shane18@hull-reynolds.com', 8306)
7 7  1a17819d berglauren@yahoo.com           916   True
8 8  fdb3837d dsmith@garcia.biz              1104  True
9 8  827fedc2 nortonjohn@davis.org           8150  False
total items: 0
[]



Must protect against unexpected errors

In [20]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(collection): # <--- ⚠
        try:
            ppr(i, el, has_next)
            # collect item if it's even
            if isinstance(el[3], int) and el[3] % 2: # < ----------------⚠
                ids.append(el[1])
            # Warn about large items
            if len(el[2]) > 20: # <-----------------------------------⚠
                print(f'!! {el}') # <-------------⚠
            if has_next == False:
                break
        except IndexError as e:
            print(f'{el} does not have 4 items')
    print(f'total items: {len(ids)}')
    return ids

ppd(process(iterate()))

0 0  e7c3a69a guerrerolisa@yahoo.com         1948  True
!! (0, 'e7c3a69a', 'guerrerolisa@yahoo.com', 1948)
1 1  e05eddbb xthomas@hayes-dixon.com        8625  True
!! (1, 'e05eddbb', 'xthomas@hayes-dixon.com', 8625)
2 2  3473a75a roberthood@houston.org         9203  True
!! (2, '3473a75a', 'roberthood@houston.org', 9203)
3 3  45b700b4 murphyangela@blankenship.org   7165  True
!! (3, '45b700b4', 'murphyangela@blankenship.org', 7165)
4 4  884186c4 christine94@aguilar.org        409   True
!! (4, '884186c4', 'christine94@aguilar.org', 409)
5 5  06bc4d16 jennifer49@ford.biz            8395  True
6 6  3186d631 heidi63@lambert.net            700   True
7 7  8ef4ed79 gdavidson@yahoo.com            9359  True
8 8  6b849134 katrinarodriguez@casey-johnson.org 3141  True
!! (8, '6b849134', 'katrinarodriguez@casey-johnson.org', 3141)
9 8  3ec7552a anthonycarson@jones.org        2572  False
!! (8, '3ec7552a', 'anthonycarson@jones.org', 2572)
total items: 7
[
  "e05eddbb",
  "3473a75a",
  "45b700b4",

---

### Let's take a step back

We use values from the raw item without knowing that they're usable

Instead of holding all the logic in this method, what if we could _ask_ each element if it was even?

In [97]:
from dataclasses import asdict, dataclass

@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int

def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(collection):
        el = Element(*el)
        print(i, el, has_next)
        # collect item if it's even
        if isinstance(el.score, int) and el.score % 2:
            ids.append(el.uuid)
        # Warn about large items
        if len(el.email) > 20:
            print(f'!! {"-".join(map(str, asdict(el).values()))}')
        if has_next == False:
            break
process(iterate())

0 Element(numeric_id=0, uuid='252231b3', email='vharvey@hotmail.com', score=6995) True
1 Element(numeric_id=1, uuid='f2e8e6f1', email='anita41@lyons.com', score=7931) True
2 Element(numeric_id=2, uuid='b4448646', email='zperez@hotmail.com', score=5693) True
3 Element(numeric_id=3, uuid='0dc91e5d', email='ashley32@gmail.com', score=5043) True
4 Element(numeric_id=4, uuid='4f3dd2a6', email='xfisher@barrett.net', score=6192) True
5 Element(numeric_id=5, uuid='1c9f26ca', email='umartinez@yahoo.com', score=7052) True
6 Element(numeric_id=6, uuid='ee79ed2c', email='linda49@rich.com', score=1944) True
7 Element(numeric_id=7, uuid='ec81942d', email='jonathanmcgee@yahoo.com', score=3751) True
!! 7-ec81942d-jonathanmcgee@yahoo.com-3751
8 Element(numeric_id=8, uuid='d289f2e1', email='jenningsjoshua@gmail.com', score=2535) True
!! 8-d289f2e1-jenningsjoshua@gmail.com-2535
9 Element(numeric_id=8, uuid='cd109e69', email='croberson@hotmail.com', score=3585) False
!! 8-cd109e69-croberson@hotmail.com-35

In [ ]:
from dataclasses import dataclass

@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int

    def is_even(self) -> bool:
        try:
            return self.score % 2
        except TypeError:
            return False

    def email_len(self, limit=20) -> bool:
        return len(self.email) > limit
    
    def as_row(self, delim='-'):
        return delim.join(map(str, [
            self.numeric_id, self.uuid, self.email, self.score,
        ]))
        
        
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(collection):
        el = Element(*el)
        print(i, el, has_next)

        if el.is_even:
            ids.append(el.uuid)
        if el.email_len() > 20:
            print(f'!! {el.as_row()}')

        if has_next == False:
            break
process(iterate())

## Now the collection itself

The collection iterator needs some work.
We need something that we can use like this:

```python
# Loop exits when no more items
for el in X:
    Element.from_api(el)
```

In [ ]:
def paginate(collection):
    for i, (el, has_next) in enumerate(collection):
        yield i, el
        if not has_next:
            return

print('---- old')
for i, el in enumerate(iterate(5)):
    if i > 8:
        break
    print(el)
    
print('---- new')
for el in paginate(iterate(5)):
    print(el)

In [ ]:
def process(collection):
    ids = []
    for i, el in paginate(collection): # <-------- ✓
        el = Element(*el)              # <-------- ⚠
        print(i, el)

        if el.is_even:
            ids.append(el.uuid)
        if el.email_len() > 20:
            print(f'!! {el.as_row()}')

process(iterate(5))

In [ ]:
@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int

    def is_even(self) -> bool:
        try:
            return self.score % 2
        except TypeError:
            return False

    def email_len(self) -> bool:
        return len(self.email)
    
    def as_row(self, delim='-'):
        return delim.join(map(str, [
            self.numeric_id, self.uuid, self.email, self.score,
        ]))
    
    @staticmethod
    def from_api(raw):
        return Element(*raw)

def paginate(collection):
    for i, (el, has_next) in enumerate(collection):
        yield i, el
        if not has_next:
            return
    
def process(collection):
    ids = []
    for i, el in paginate(collection):
        el = Element.from_api(el)
        print(i, el)

        if el.is_even:
            ids.append(el.uuid)
        if el.email_len() > 20:
            print(f'!! {el.as_row()}')

process(iterate(5))

What if we want to send all the even and odd records to different places?
Or, collect all the emails from both categories?

In [ ]:
def process(collection, debug=False):
    even_ids = []
    odd_ids = []
    for i, el in paginate(collection):
        el = Element.from_api(el)
        print(i, el)

        if el.is_even():
            even_ids.append(el.uuid)
        else:
            odd_ids.append(el.uuid)
        if el.email_len() > 20 and debug:
            print(f'!! {el.as_row()}')
    return even_ids, odd_ids

even, odd = process(iterate(8))
print('\n', 'even:', len(even), 'odd:', len(odd))
print(even)

What if we want to collect the emails of odd/even people instead? or something else in the future?

Step 1: just return the entire objects, don't grab values from them

In [ ]:
def process(collection, debug=False):
    even = []
    odd = []
    for i, el in paginate(collection):
        el = Element.from_api(el)
        print(i, el)

        if el.is_even():
            even.append(el)
        else:
            odd.append(el)
        if el.email_len() > 20 and debug:
            print(f'!! {el.as_row()}')
    return even, odd

even, odd = process(iterate(8))
print('\n', 'even:', len(even), 'odd:', len(odd))
print([el.email for el in even])

In [ ]:
from collections import Counter
from typing import List
from itertools import filterfalse

def paginate(collection):
    for el, has_next in collection:
        yield el
        if not has_next:
            return

@dataclass
class Collection:
    items: List[Element]

    def from_raw(items):
        return Collection(list(map(Element.from_api, items)))
        
    def emails(self):
        return [el.email for el in self.items]

    def __iter__(self):
        yield from self.items
        
    def odd_records(self):
        return Collection(list(filter(lambda x: x.score % 2, self.items)))
    
    def even_records(self):
        return Collection(list(filterfalse(lambda x: x.score % 2, self.items)))


c = Collection.from_raw(paginate(iterate(8)))
print('all emails\n', c.emails())

print('\nodd records\n', c.odd_records())
print('\neven records\n', c.even_records())

print('\neven emails!\n', c.even_records().emails())

In [21]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('8bZh5LMaSmE?t=350')

---

### Filtering and sorting

If you have a static method (not an instance method), you can filter with that instead of having to use a lambda

In [ ]:
@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int

    @staticmethod
    def from_api(raw):
        return Element(*raw)
        
    def is_even(self) -> bool:
        try:
            return self.score % 2
        except TypeError:
            return False
    
    @staticmethod
    def _is_even(element):
        return element._is_even()
    
    @staticmethod
    def _is_false(element):
        return not element._is_even()

    def email_len(self) -> bool:
        return len(self.email)
    
    def as_row(self, delim='-'):
        return delim.join(map(str, [
            self.numeric_id, self.uuid, self.email, self.score,
        ]))

@dataclass
class Collection:
    items: List[Element]

    def from_raw(items):
        return Collection(list(map(Element.from_api, items)))
        
    @property
    def emails(self):
        return [el.email for el in self.items]

    def __iter__(self):
        yield from self.items
        
    def filter_records(self, pred):
        return Collection(list(filter(pred, self.items)))


c = Collection.from_raw(paginate(iterate()))
print('total items', len(c.items))

even = c.filter_records(Element.is_even)
print('\neven items\n', len(even.emails), even.emails)

odd = c.filter_records(Element.is_even)
print('\nodd items\n', len(even.emails), even.emails)

In [ ]:
c.filter_records(lambda x: x.email.startswith('a'))

In [ ]:
c.filter_records(lambda x: '@' in x.email)

In [ ]:
@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int

    @staticmethod
    def from_api(raw):
        return Element(*raw)
        
    def is_even(self) -> bool:
        try:
            return self.score % 2
        except TypeError:
            return False
    
    @staticmethod
    def _is_even(element):
        return element._is_even()
    
    @staticmethod
    def _is_false(element):
        return not element._is_even()

    def email_len(self) -> bool:
        return len(self.email)
    
    def as_row(self, delim='-'):
        return delim.join(map(str, [
            self.numeric_id, self.uuid, self.email, self.score,
        ]))

@dataclass
class Collection:
    items: List[Element]

    def from_raw(items):
        return Collection(list(map(Element.from_api, items)))

    @property
    def emails(self):
        return [el.email for el in self.items]

    def __iter__(self):
        yield from self.items

    def filter_records(self, pred):
        return Collection(list(filter(pred, self.items)))

    
c = Collection.from_raw(paginate(iterate()))
print('total items', len(c.items))

even = c.filter_records(Element.is_even)
print('\neven items\n', len(even.emails), even.emails)

odd = c.filter_records(Element.is_even)
print('\nodd items\n', len(even.emails), even.emails)

## Representation

Dunder methods!

Let's make a few options:

- All objects as JSON
- All objects as rows/lists

In [ ]:
@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int

    @staticmethod
    def from_api(raw):
        return Element(*raw)
        
    def is_even(self) -> bool:
        try:
            return self.score % 2
        except TypeError:
            return False
    
    @staticmethod
    def _is_even(element):
        return element._is_even()
    
    @staticmethod
    def _is_false(element):
        return not element._is_even()

    def email_len(self) -> bool:
        return len(self.email)
    
    def as_row(self, delim='-'):
        return delim.join(map(str, [
            self.numeric_id, self.uuid, self.email, self.score,
        ]))
    
    def as_json(self):
        return json.dumps(self.__dict__)

@dataclass
class Collection:
    items: List[Element]

    def from_raw(items):
        return Collection(list(map(Element.from_api, items)))
        
    @property
    def emails(self):
        return [el.email for el in self.items]

    def __iter__(self):
        yield from self.items
        
    def filter_records(self, pred):
        return Collection(list(filter(pred, self.items)))
    
    def as_json(self, **kwargs):
        return json.dumps(
            [asdict(el) for el in self.items],
            **kwargs
        )

    
c = Collection.from_raw(paginate(iterate()))
print('total items', len(c.items))

even = c.filter_records(Element.is_even)
print('\neven items\n', len(even.emails), even.emails)

odd = c.filter_records(Element.is_even)
print('\nodd items\n', len(even.emails), even.emails)

In [ ]:
c = Collection.from_raw(paginate(iterate()))

ppj(c.items[0].as_json())
ppj(c.as_json())

In [ ]:
ppj(c[:2].as_json(indent=2))

In [ ]:
What if we just want to print the first few items?

In [ ]:
What if we just want to print the first few items

In [ ]:
ppj(c.items[:2].as_json(indent=2))

In [ ]:
ppj(Collection(c.items[:2]).as_json(indent=2))

In [ ]:
@dataclass
class Collection:
    items: List[Element]

    def from_raw(items):
        return Collection(list(map(Element.from_api, items)))

    def __getitem__(self, i):
        return Collection(self.items[i])
    
    @property
    def emails(self):
        return [el.email for el in self.items]

    def __iter__(self):
        yield from self.items
        
    def filter_records(self, pred):
        return Collection(list(filter(pred, self.items)))
    
    def as_json(self, **kwargs):
        return json.dumps(
            [asdict(el) for el in self.items],
            **kwargs
        )

In [ ]:
c = Collection.from_raw(paginate(iterate()))

ppj(c[1:2].as_json(indent=2))
ppj(c[1:3].as_json(indent=2))

### Sorting!

You can sort easily if you already have handy methods available for getting the values to sort by

In [ ]:
list(sorted(c, key=lambda x: x.score))

https://github.com/tomquirk/realestate-com-au-api/blob/8368da02a67aaf1c2fe9634f19181fb54685718d/realestate_com_au/realestate_com_au.py#L70-L118